In [ ]:
%pip install adal

In [ ]:
## Example of using the MD Sync REST API, using Service Prinipal

import msal
import requests
import adal

 
def get_sqlendpoint(workspaceId, header,lakehouse_name):
    """
    This list the SQL endpoints for the given workspace.
    """
    getsqlendpoint = f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/sqlEndpoints'
    # Make the GET request
    response = requests.get(url=getsqlendpoint, headers=header)
    # Extract the list of SQL endpoints from the response and find the SQl Endoint ID for the specfied lakehouse
    # git_status = response.json()
    # print(git_status)
    get_sqlenpoints=response.json().get('value', [])
    for sqlendpoint in get_sqlenpoints:
        if sqlendpoint['displayName'].lower() == lakehouse_name.lower():
            sqlendpoint_id = sqlendpoint['id']
            return(sqlendpoint_id)
def refresh_sql_endpoint(sqlendpoint_id, header,workspaceId):
    """
    Calls the SQL Endpoint refresh API for the specified SQL Endpoint ID.
    """
    #refresh_url = f'https://api.powerbi.com/v1/workspaces/{workspaceId}/sqlEndpoints/{sqlendpoint_id}/refreshMetadata?preview=true' #preview=true is not needed anymore
    # The URL for the SQL Endpoint refresh API
    refresh_url = f'https://api.powerbi.com/v1/workspaces/{workspaceId}/sqlEndpoints/{sqlendpoint_id}/refreshMetadata'
    print(refresh_url)
    # Define the payload for the refresh command
    # The payload is a JSON object that specifies the command to refresh the SQL Endpoint
    payload = {} 
    # Make the POST request to refresh the SQL Endpoint
    response = requests.post(url=refresh_url, headers=header,json=payload)
    if response.status_code == 202:
        print(f"SQL Endpoint with ID '{sqlendpoint_id}' is being refreshed.")
    else:
        print(f"Status Code: {response.status_code}, Response: {response.text}")
        
if __name__ == "__main__":
    #TODO: Replace the below to code to pull from information from Key Vault.
    #https://learn.microsoft.com/en-us/azure/key-vault/secrets/quick-create-python?tabs=azure-cli
    TENANT_ID = '__add the tenant id__'
    CLIENT_ID = '__add the client id__'
    CLIENT_SECRET = '__add the client secret__'
    workspaceId = '__put_your_workspace_here__'
    lakehouse_name='__put_your_lakehouse_here__'
    # Azure Resource Manager for your Azure subscription
    RESOURCE = 'https://analysis.windows.net/powerbi/api'
    # The authority URL
    AUTHORITY_URL = f'https://login.microsoftonline.com/{TENANT_ID}'
    # The scope for the token
    SCOPE = ['https://api.fabric.microsoft.com/.default']

    # Create a public client application
    app = msal.ConfidentialClientApplication(
        client_id=CLIENT_ID,
        client_credential=CLIENT_SECRET,
        authority=AUTHORITY_URL
    )
    result = app.acquire_token_for_client(scopes=SCOPE)
    if 'access_token' in result:
        access_token = result['access_token']
    else:
        raise Exception(f"Failed to acquire token: {result.get('error_description')}")

    header = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }
    sqlendpoint_id = get_sqlendpoint(workspaceId, header, lakehouse_name)
    refresh_sql_endpoint(sqlendpoint_id, header,workspaceId)